<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла.

In [ ]:
df = pd.read_csv(
    './data/dst-3.0_16_1_hh_database.csv',
    sep=';',
    encoding='utf-8',
)

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
df.head(2)

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

In [ ]:
df.info()

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
df.count()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
df.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
def extract_education_level(text):
    # Берём первые 2 слова
    words = text.split()[:2]
    # Собираем в строку
    prefix = ' '.join(words).lower()

    if 'высшее' in prefix and 'неоконченное' in prefix:
        return 'неоконченное высшее'
    elif 'высшее' in prefix:
        return 'высшее'
    elif 'среднее специальное' in prefix:
        return 'среднее специальное'
    elif 'среднее' in prefix:
        return 'среднее'
    else:
        return 'неизвестно'

df['Образование'] = df['Образование и ВУЗ'].apply(extract_education_level)

In [ ]:
# Проверяем уникальные значения
df['Образование'].unique()

In [ ]:
df['Образование'].value_counts()

In [ ]:
# Удаляем старый столбец
df.drop('Образование и ВУЗ', axis=1, inplace=True)

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина.
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '.
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
def extract_sex(text):
     if 'Мужчина' in text.split()[:1]:
         return 'М'
     else:
         return 'Ж'

def extract_age(text):
    parts = text.split(' , ')
    age_part = parts[1].strip()
    age_str = age_part.split()[0]
    return int(age_str)

df['Пол'] = df['Пол, возраст'].apply(extract_sex)
df['Возраст'] = df['Пол, возраст'].apply(extract_age)
df.drop('Пол, возраст', axis=1, inplace=True)

In [ ]:
df['Пол'].value_counts()

In [ ]:
df['Возраст'].mean()

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**.

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
def extract_experience_months(text):
    if pd.isna(text) or text == "Не указано":
        return np.nan

    words = text.split()

    try:
        start = 2
    except (ValueError, IndexError):
        return np.nan

    segment = words[start:start + 4]

    years = 0
    months = 0

    i = 0
    while i < len(segment):
        word = segment[i]
        if word.isdigit():
            num = int(word)
            if i + 1 < len(segment):
                unit = segment[i + 1]
                if unit.startswith('год') or unit in ('лет', 'года'):
                    years = num
                elif unit.startswith('месяц'):
                    months = num
                i += 2
            else:
                i += 1
        else:
            i += 1

    return years * 12 + months

In [ ]:
df['Опыт работы (месяц)'] = df['Опыт работы'].apply(extract_experience_months)

In [ ]:
df['Опыт работы (месяц)'].median()

In [ ]:
df.drop('Опыт работы', axis=1, inplace=True)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , …
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
million_cities = [
    'Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск',
    'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск',
    'Пермь', 'Воронеж', 'Волгоград'
]

def extract_city(text):
    parts = [part.strip() for part in text.split(' , ')]
    city_candidate = parts[0]

    if city_candidate == 'Москва':
        return 'Москва'
    elif city_candidate == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    elif city_candidate in million_cities:
        return 'город-миллионник'
    else:
        return 'другие'

In [ ]:
def is_ready_to_relocate(text):
    text_lower = text.lower()
    if 'не готов к переезду' in text_lower or 'не готова к переезду' in text_lower:
        return False
    return True

In [ ]:
def is_ready_for_business_trips(text):
    text_lower = text.lower()
    if 'не готов к командировкам' in text_lower or 'не готова к командировкам' in text_lower:
        return False
    return True

In [ ]:
df['Город'] = df['Город, переезд, командировки'].apply(extract_city)
df['Готовность к переезду'] = df['Город, переезд, командировки'].apply(is_ready_to_relocate)
df['Готовность к командировкам'] = df['Город, переезд, командировки'].apply(is_ready_for_business_trips)

In [ ]:
df.drop('Город, переезд, командировки', axis=1, inplace=True)

In [ ]:
# Проценты по всем городам
df['Город'].value_counts(normalize=True) * 100

In [ ]:
# Сколько процентов соискателей готовы одновременно и к переездам, и к командировкам
(df['Готовность к переезду'] & df['Готовность к командировкам']).mean() * 100

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
# Список нужных категорий
employment_types = [
    'полная занятость',
    'частичная занятость',
    'проектная работа',
    'волонтерство',
    'стажировка']

# Создаём новые столбцы
for cat in employment_types:
    df[cat] = df['Занятость'].str.contains(cat)

In [ ]:
schedule_types = [
    'полный день',
    'сменный график',
    'гибкий график',
    'удаленная работа',
    'вахтовый метод'
]
for cat in schedule_types:
    df[cat] = df['График'].str.contains(cat)

In [ ]:
df.drop('Занятость', axis=1, inplace=True)
df.drop('График', axis=1, inplace=True)

In [ ]:
# Сколько людей ищут проектную работу и волонтерство (в обоих столбцах стоит True)?
(df['проектная работа'] & df['волонтерство']).sum()

In [ ]:
# Сколько людей хотят работать вахтовым методом и с гибким графиком (в обоих столбцах стоит True)?
(df['вахтовый метод'] & df['гибкий график']).sum()

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**.
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата,
* "proportion" - пропорция,
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования:
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# Преобразуем в datetime с указанием формата
df['Обновление резюме'] = pd.to_datetime(df['Обновление резюме'], format='%d.%m.%Y %H:%M')

In [ ]:
# Оставляем только дату (без времени)
df['Обновление резюме'] = df['Обновление резюме'].dt.date

In [ ]:
# проверка формата
type(df['Обновление резюме'].iloc[0])

In [ ]:
currency_map = {
    'руб.': 'RUB',
    'USD': 'USD',
    'KZT': 'KZT',
    'грн.': 'UAH',
    'EUR': 'EUR',
    'KGS': 'KGS',
    'сум': 'UZS',
    'AZN': 'AZN',
    'бел.руб.': 'BYN',
}
def extract_salary_and_currency(text):
    parts = text.split()

    amount_str = parts[0]
    amount = int(amount_str)

    # Последнее слово — валюта
    currency_raw = parts[-1]
    iso_currency = currency_map.get(currency_raw)

    return amount, iso_currency

In [ ]:
# Применяем функцию и создаём два новых столбца
df[['ЗП (число)', 'Валюта']] = df['ЗП'].apply(
    lambda x: pd.Series(extract_salary_and_currency(x))
)

In [ ]:
df['Валюта'].info()

In [ ]:
# загружаем новые данные по курсам валют
df_currency = pd.read_csv('./data/ExchangeRates.csv')

In [ ]:
# Преобразуем в datetime с указанием формата
df_currency['date'] = pd.to_datetime(df_currency['date'], format='%d/%m/%y')

In [ ]:
# Оставляем только нужные столбцы
columns_to_keep = ['currency', 'date', 'proportion', 'close']
df_currency = df_currency[columns_to_keep]

In [ ]:
df_currency['date'] = pd.to_datetime(df_currency['date'])

In [ ]:
# выравниваем формат даты
df['Обновление резюме'] = pd.to_datetime(df['Обновление резюме'], errors='coerce')

In [ ]:
# проверка формата
print(df['Обновление резюме'].dtype)
print(df_currency['date'].dtype)

In [ ]:
# Объединение таблиц
# LEFT JOIN
df = df.merge(
    df_currency,
    how='left',
    left_on=['Валюта', 'Обновление резюме'],
    right_on=['currency', 'date']
)

In [ ]:
# Заполняем курс = 1 для всех строк с валютой RUB
df.loc[df['Валюта'] == 'RUB', 'close'] = 1.0

In [ ]:
# Заполняем proportion = 1 для всех строк с валютой RUB
df.loc[df['Валюта'] == 'RUB', 'proportion'] = 1.0

In [ ]:
# Умножить сумму желаемой заработной платы на присоединённый курс валюты (close) и разделить на пропорцию. Обратите внимание на пропуски после объединения в этих столбцах. Результат занести в новый столбец «ЗП (руб)»
df['ЗП (руб)'] = (df['ЗП (число)'] * df['close']) / df['proportion']

In [ ]:
# Чему равна желаемая медианная заработная плата соискателей в нашей таблице (в рублях)?
df['ЗП (руб)'].median()

In [ ]:
# 4. Удаление временных столбцов
df.drop(columns=['close', 'date', 'proportion', 'currency', 'ЗП', 'ЗП (число)', 'Валюта'], inplace=True)

In [ ]:
# проверяем итоговую таблицу
df.info()

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig1 = px.histogram(
    df,
    x='Возраст',
    nbins=500,
    title='Гистограмма возраста соискателей',
    labels={'Возраст': 'Возраст', 'count': 'Количество'},
    color_discrete_sequence=['steelblue']
)
fig1.show()

fig2 = px.box(
    df,
    y='Возраст',
    title='Коробчатая диаграмма возраста соискателей',
    labels={'Возраст': 'Возраст'},
    color_discrete_sequence=['lightcoral'],
    height=600
)
fig2.show()

In [ ]:
# Чему равно модальное значение возраста соискателей?
mode_age = df['Возраст'].mode().tolist()
print("Мода распределения возраста:", mode_age)

In [ ]:
print("Минимум:", df['Возраст'].min())
print("Максимум:", df['Возраст'].max())
print("25-й перцентиль:", df['Возраст'].quantile(0.25))
print("Медиана:", df['Возраст'].median())
print("75-й перцентиль:", df['Возраст'].quantile(0.75))

In [ ]:
Q1 = df['Возраст'].quantile(0.25)
Q3 = df['Возраст'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Возраст'] < lower_bound) | (df['Возраст'] > upper_bound)]
print(f"Количество выбросов: {len(outliers)}")
print("Примеры аномальных возрастов:")
print(outliers['Возраст'].sort_values().unique())

1. Чему равна мода распределения?\
Мода = 30 лет — это наиболее часто встречающийся возраст среди соискателей.

2. Каковы предельные значения признака?\
Минимальный возраст: 14 лет\
Максимальный возраст: 100 лет

3. В каком примерном интервале находится возраст большинства соискателей?\
Большинство соискателей (50%) находятся в возрасте от 27 до 36 лет\
(это интерквартильный размах: 25-й перцентиль = 27, 75-й перцентиль = 36).\
Типичный возрастной диапазон: 27–36 лет.

4. Есть ли аномалии для признака? Какие значения вы бы причислили к их числу?\
Aномалии есть.\
По правилу Тьюки (IQR-метод) аномальными считаются значения менее 19.5 или более 43.5 лет.\
Верхние выбросы: от 50 до 100 лет (включая 50, 60, 70, 76, 100 и др.)\
Нижние выбросы: возраст < 19.5 → например, 14–19 лет (в данных таких 14 человек)\
Однако исходя из здравого смысла выбросами можно считать только значения 76, 77, 100 лет - это могут быть ошибки ввода.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig1 = px.histogram(
    df,
    x='Опыт работы (месяц)',
    nbins=1500,
    title='Гистограмма опыта работы (в месяцах)',
    labels={'Опыт работы (месяц)': 'Опыт (месяцы)', 'count': 'Количество'},
    color_discrete_sequence=['orange']
)
fig1.show()

fig2 = px.box(
    df,
    y='Опыт работы (месяц)',
    title='Коробчатая диаграмма опыта работы (в месяцах)',
    labels={'Опыт работы (месяц)': 'Опыт (месяцы)'},
    color_discrete_sequence=['lightcoral'],
    height=700
)
fig2.show()

In [ ]:
df['Опыт работы (месяц)'].mode().tolist()

In [ ]:
# Чему равен максимальный опыт работы (в месяцах)?
print("Максимум:", df['Опыт работы (месяц)'].max())

In [ ]:
print("Минимум:", df['Опыт работы (месяц)'].min())
print("Максимум:", df['Опыт работы (месяц)'].max())
print("25-й перцентиль:", df['Опыт работы (месяц)'].quantile(0.25))
print("Медиана:", df['Опыт работы (месяц)'].median())
print("75-й перцентиль:", df['Опыт работы (месяц)'].quantile(0.75))

In [ ]:
Q1 = df['Опыт работы (месяц)'].quantile(0.25)
Q3 = df['Опыт работы (месяц)'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

outliers = df[df['Опыт работы (месяц)'] > upper_bound]
print(f"Количество выбросов: {len(outliers)}")
print("Максимальные значения:")
print(outliers['Опыт работы (месяц)'].sort_values(ascending=False).head(10).values)

1. Чему равна мода распределения?\
Мода = 81 месяц\
Это наиболее частый общий стаж среди соискателей (≈ 6 лет 9 месяцев).

2. Каковы предельные значения признака?\
Минимальный опыт: 1 месяц\
Максимальный опыт: 1188 месяцев

3. В каком примерном интервале находится опыт работы большинства соискателей?\
Большинство соискателей (50%) имеют опыт от 57 до 154 месяцев

4. Есть ли аномалии для признака? Какие значения вы бы причислили к их числу?\
Аномалии есть.
По правилу Тьюки (IQR-метод), верхняя граница нормы =154 + 1.5 × (154 − 57) = 154 + 145.5 = 299.5 месяцев (~25 лет).\
Значит, все значения > 299.5 месяцев — могут быть выбросами.\
Но исходя из здравого смысла к аномалиям можно отнести:\
Опыт ≥ 300 месяцев (≥ 25 лет) — маловероятен для соискателей при возрасте 20–40 лет.\
Особенно подозрительны:\
556–663 месяцев → 46–55 лет опыта\
1188 месяцев → 99 лет опыта — явная ошибка в данных или опечатка.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
# Гистограмма
fig1 = px.histogram(
    x=df['ЗП (руб)'],
    nbins=50,
    title='Гистограмма желаемой ЗП (в рублях)',
    labels={'x': 'ЗП (руб)', 'y': 'Количество'},
    color_discrete_sequence=['steelblue']
)
fig1.show()

# Boxplot
fig2 = px.box(
    y=df['ЗП (руб)'],
    title='Коробчатая диаграмма ЗП (в рублях)',
    labels={'y': 'ЗП (руб)'},
    color_discrete_sequence=['orange']
)
fig2.show()

In [ ]:
# эта отображает лучше
df['log10_ЗП'] = np.log10(df['ЗП (руб)'])

fig = px.histogram(
    df,
    x='log10_ЗП',
    nbins=1500,
    title='Гистограмма желаемой ЗП log10 (в рублях)',
    labels={'log10_ЗП': 'log10(ЗП, руб)', 'count': 'Количество'},
    color_discrete_sequence=['orange']
)
fig.show()
# Удаляем временный столбец
df.drop('log10_ЗП', axis=1, inplace=True)

In [ ]:
print("Минимум:", df['ЗП (руб)'].min())
print("Максимум:", df['ЗП (руб)'].max())
print("25-й перцентиль:", df['ЗП (руб)'].quantile(0.25))
print("Медиана:", df['ЗП (руб)'].median())
print("75-й перцентиль:", df['ЗП (руб)'].quantile(0.75))

In [ ]:
Q1 = df['ЗП (руб)'].quantile(0.25)
Q3 = df['ЗП (руб)'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

outliers = df[df['ЗП (руб)'] > upper_bound]
print(f"Количество выбросов: {len(outliers)}")
print("Максимальные ЗП (руб):")
print(outliers['ЗП (руб)'].sort_values(ascending=False).head(10).values)

In [ ]:
# Количество соискателей с ЗП > 1 000 000 рублей
count_over_1m = (df['ЗП (руб)'] > 1_000_000).sum()
print(count_over_1m)

1. Каковы предельные значения признака «ЗП (руб)»?\
Минимальная желаемая зарплата: 1 рубль\
Максимальная желаемая зарплата: 24 304 876 рублей\
Значение в 24+ млн рублей — это экстремальный выброс.\

2. В каком примерном интервале находится заработная плата большинства соискателей?\
Из перцентилей:\
25-й перцентиль: 37 082 руб.\
75-й перцентиль: 95 000 руб.\
Большинство соискателей (50%) указывают желаемую зарплату в диапазоне от ~37 000 до ~95 000 рублей.\
Медиана = 59 019 руб.

3. Есть ли аномалии? Обратите внимание на гигантские размеры желаемой заработной платы.
Есть значительные аномалии.\
Наиболее экстремальные значения:\
24 304 876 руб.\
7 675 224 руб.\
3 000 000 руб.\
2 500 000 руб.\
и далее значения до 750 000 руб.\
Зарплаты > 1 000 000 руб. встречаются редко в реальности.
Значения в миллионы рублей в сочетании с обычными должностями (например, "менеджер", "специалист") почти наверняка ошибки.
Также выбросом можно считать зарплату в 1 рубль (минимум).

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
# Фильтрация: ЗП < 1 000 000 руб.
df_filtered = df[df['ЗП (руб)'] < 1_000_000]

In [ ]:
# медиана по уровню образования
median_salary_by_education = df_filtered.groupby('Образование')['ЗП (руб)'].median().sort_values(ascending=False)
print(median_salary_by_education)

In [ ]:
plot_df = median_salary_by_education.reset_index()
plot_df.columns = ['Уровень образования', 'Медианная ЗП (руб)']

fig = px.bar(
    plot_df,
    x='Уровень образования',
    y='Медианная ЗП (руб)',
    color='Уровень образования',
    color_discrete_sequence=px.colors.sequential.Viridis,
    title='Медианная желаемая ЗП по уровню образования (ЗП < 1 млн руб)',
    text='Медианная ЗП (руб)'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_layout(showlegend=False)
fig.show()

1. Наибольшие и наименьшие уровни желаемой заработной платы\
Наибольший уровень желаемой заработной платы у людей получивших высшее образование.\
Наименьший уровень желаемой заработной платы у людей со средним и средне специальным образованием.\
Таким образом, существует чёткая положительная связь между уровнем образования и медианной желаемой заработной платой.

2. Важен ли признак уровня образования при прогнозировании заработной платы?\
чем выше уровень образования, тем выше ожидаемая ЗП.\
Это согласуется с общепринятой практикой на рынке труда: более высокий уровень образования часто ассоциируется с более сложными, квалифицированными и высокооплачиваемыми ролями.\
Уровень образования важен при прогнозировании заработной платы.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
city_order = (
    df_filtered.groupby('Город')['ЗП (руб)']
    .median()
    .sort_values(ascending=False)
    .index
)

fig = px.box(
    df_filtered,
    x='Город',
    y='ЗП (руб)',
    color='Город',
    category_orders={'Город': city_order},
    title='Распределение желаемой ЗП по городам (ЗП < 1 млн руб)',
    labels={'ЗП (руб)': 'Желаемая ЗП (руб)', 'Город': 'Город'},
    color_discrete_sequence=px.colors.qualitative.Prism
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_layout(showlegend=False, height=1000)

fig.show()

In [ ]:
# Медиана и IQR по городам
stats = df_filtered.groupby('Город')['ЗП (руб)'].agg(
    median='median',
    q25=lambda x: x.quantile(0.25),
    q75=lambda x: x.quantile(0.75)
).round().astype(int)

stats['IQR'] = stats['q75'] - stats['q25']
stats = stats.sort_values('median', ascending=False)
print(stats)

1. Соотношение медианных уровней желаемой заработной платы\
Москва имеет наивысшую медианную ЗП - 85 000 руб.\
Это значительно выше, чем в других городах: почти в 1.4 раза выше Санкт-Петербурга и в 2.1 раза выше городов-миллионников и "других".\
Санкт-Петербург на втором месте с медианой 60 000 руб., что логично - это второй по значимости город в России.\
Города-миллионники и "другие" имеют одинаковую медиану - 40 000 руб., что указывает на схожесть структуры вакансий в них.
2. Размах зарплат
Москва: IQR = 90 000 руб. - самый широкий размах.\
Это говорит о высокой дифференциации рынка.\
Санкт-Петербург: IQR ≈ 40 000 руб. — умеренный размах.\
Города-миллионники и "другие": IQR = 30 000 руб. - наименьший размах, что указывает на однородность рынка труда.
3. Важен ли признак 'Город' при прогнозировании заработной платы?
Да, признак 'Город' является важным предиктором заработной платы.\
Наблюдается чёткая иерархия по медианной ЗП:\
Москва > Санкт-Петербург > регионы.\
Разница в медианах статистически значима и экономически существенна.\
Размах зарплат также зависит от города, что говорит о разных рыночных условиях.\
Город косвенно отражает:стоимость жизни, уровень спроса на специалистов и др.\
Даже при одинаковом опыте, образовании и должности, резюме из Москвы будет иметь значительно более высокую желаемую ЗП, чем из региона.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
# Агрегация: медиана ЗП по комбинациям двух признаков
grouped = df_filtered.groupby(['Готовность к переезду', 'Готовность к командировкам'])['ЗП (руб)'].median().reset_index()
print(grouped)

# Преобразуем булевы в строки для удобства отображения
grouped['Готовность к переезду'] = grouped['Готовность к переезду'].map({True: 'Готов', False: 'Не готов'})
grouped['Готовность к командировкам'] = grouped['Готовность к командировкам'].map({True: 'Готов', False: 'Не готов'})

In [ ]:
# Фильтрация: готовы и к переезду, и к командировкам
filtered = df[(df['Готовность к переезду'] == True) & (df['Готовность к командировкам'] == True)]
# Медианная ЗП в рублях
filtered['ЗП (руб)'].median()

In [ ]:
fig = px.bar(
    grouped,
    x='Готовность к переезду',
    y='ЗП (руб)',
    color='Готовность к командировкам',
    barmode='group',
    title='Медианная ЗП в зависимости от готовности к переезду и командировкам<br>(ЗП < 1 млн руб.)',
    labels={
        'ЗП (руб)': 'Медианная ЗП (руб)',
        'Готовность к переезду': 'Готовность к переезду'
    },
    color_discrete_sequence=px.colors.qualitative.Pastel1,
    text='ЗП (руб)'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

fig.update_layout(
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='lightgray'),
    xaxis=dict(tickangle=0),
    height=600,
    width=800
)

fig.show()

1. Проанализируйте, сравнив уровень заработной платы в категориях.\
Оба признака - 'Готовность к переезду' и 'Готовность к командировкам - значительно влияют на уровень желаемой заработной платы.\
Готовность к командировкам оказывает более сильное влияние на уровень ЗП, чем готовность к переезду.\
Однако готовность к командировкам является более сильным фактором, чем готовность к переезду.\
Самая низкая ЗП - у тех, кто не готов ни к переезду, ни к командировкам

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
# Создание возрастной группы
df['Возрастная группа'] = pd.cut(
    df['Возраст'],
    bins=[17, 25, 30, 35, 40, 50, 100],
    labels=['18–25', '26–30', '31–35', '36–40', '41–50', '50+'],
    right=True
)

pivot = df.pivot_table(
    values='ЗП (руб)',
    index='Возрастная группа',
    columns='Образование',
    aggfunc='median',
    observed=False
)

pivot_long = pivot.reset_index().melt(
    id_vars='Возрастная группа',
    var_name='Образование',
    value_name='ЗП (руб)'
)

pivot_long = pivot_long.dropna()

fig = px.density_heatmap(
    pivot_long,
    x='Образование',
    y='Возрастная группа',
    z='ЗП (руб)',
    color_continuous_scale='YlGnBu',
    text_auto='.0f',
    title='Медианная ЗП по возрасту и образованию',
    labels={'ЗП (руб)': 'Медианная ЗП (руб)'}
)

fig.update_yaxes(categoryorder='array', categoryarray=['18–25', '26–30', '31–35', '36–40', '41–50', '50+'])
fig.show()

In [ ]:
# Удаляем временный столбец
df.drop('Возрастная группа', axis=1, inplace=True)

1. Проанализируйте тепловую карту, сравнив показатели внутри групп.\
При любом возрасте медианная ЗП выше у соискателей с высшим образованием, чем у остальных.\
У всех групп наблюдается рост ЗП с возрастом до группы 36-40 лет\
Плато или спад после 40 лет.\
Образование это ключевой фактор, определяющий как абсолютный уровень ЗП, так и другие карьерные возможности.\
Возраст усиливает эффект образования - чем старше соискатель, тем шире разрыв между уровнями ЗП по образованию.\
Пик карьеры примерно 36–50 лет, после чего ЗП может стабилизироваться или падать, особенно у соискателей с низким уровнем образования.\
При прогнозировании ЗП оба признака - возраст и образование критически важны.\
При анализе заработной платы нельзя рассматривать возраст или образование изолированно - их влияние взаимоусиливающее.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
# Преобразуем опыт из месяцев в годы (с плавающей точкой для точности)
df['Опыт (лет)'] = df['Опыт работы (месяц)'] / 12.0

# Оставим только строки с валидным опытом
df_plot = df.dropna(subset=['Опыт (лет)', 'Возраст'])

In [ ]:
fig = px.scatter(
    df_plot,
    x='Возраст',
    y='Опыт (лет)',
    opacity=0.6,
    labels={
        'Возраст': 'Возраст (лет)',
        'Опыт (лет)': 'Опыт работы (лет)'
    },
    title='Зависимость опыта работы от возраста',
    color_discrete_sequence=['orange']
)

line_df = pd.DataFrame({'x': [0, 100], 'y': [0, 100]})
fig.add_trace(px.line(line_df, x='x', y='y', color_discrete_sequence=['red']).data[0])
fig.data[-1].name = 'Опыт = Возраст'

fig.update_xaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [ ]:
# опыт > возраста (строго выше прямой y = x)
above_line = df[df['Опыт (лет)'] > df['Возраст']]
num_above = len(above_line)
print(f"Количество точек строго выше прямой y = x: {num_above}")

In [ ]:
# Удаляем временный столбец
df.drop('Опыт (лет)', axis=1, inplace=True)

Диаграмма наглядно показывает реалистичные пары (возраст, опыт) — все они ниже прямой y = x.\
Аномалии - это точки на или выше красной линии, их стоит проверить или удалить.\
На полученной диаграмме аномалии есть, но их довольно мало.


**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
df_agg = df_filtered.groupby(['Образование', 'Город'])['ЗП (руб)'].median().reset_index()

fig = px.bar(
    df_agg,
    x='Образование',
    y='ЗП (руб)',
    color='Город',
    barmode='group',
    title='Медианная ЗП по образованию и городу',
    labels={'ЗП (руб)': 'Медианная ЗП (руб)', 'Образование': 'Уровень образования'},
    color_discrete_sequence=px.colors.qualitative.Set1,
    category_orders={
        'Образование': ['среднее', 'среднее специальное', 'неоконченное высшее', 'высшее']
    }
)

fig.update_layout(
    yaxis_title='Медианная ЗП (руб)',
    xaxis_title='Уровень образования',
    legend_title='Город',
    height=600,
    margin=dict(l=40, r=40, t=80, b=100)
)

fig.show()

Высшее образование и Москва это самая высокая ЗП (~85–90 тыс. руб.)\
В «других» городах даже с высшим образованием ЗП в 2 раза ниже, чем в Москве.\
Среднее образование почти не зависит от города - везде 35–45 тыс. руб.\
Санкт-Петербург занимает промежуточное положение между Москвой и регионами.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их.

In [ ]:
# Полные дубликаты
num_duplicates = df.duplicated().sum()
print(f"Найдено дубликатов: {num_duplicates}")

# Удаление полных дубликатов
df_clean = df.drop_duplicates().reset_index(drop=True)

# Проверка
print(f"Размер до удаления: {len(df)}")
print(f"Размер после удаления: {len(df_clean)}")

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах.

In [ ]:
df_clean.isnull().sum()

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# Удаляем строки с пропусками в столбцах с местом работы и должностью
df_clean = df_clean.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])

# Проверяем
print(f"Количество строк: {len(df_clean)}")

In [ ]:
# Заполняем пропуски в столбце "Опыт работы (месяц)" медианным значением
median_experience = df_clean['Опыт работы (месяц)'].median()
df_clean['Опыт работы (месяц)'] = df_clean['Опыт работы (месяц)'].fillna(median_experience)

# Проверяем
print("Количество пропусков по столбцам:")
print(df_clean[['Опыт работы (месяц)', 'Последнее/нынешнее место работы', 'Последняя/нынешняя должность']].isnull().sum())

In [ ]:
# результирующее среднее значение в столбце «Опыт работы (месяц)» после заполнения пропусков
round(df_clean['Опыт работы (месяц)'].mean())

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# смотрим количество выбросов
((df_clean['ЗП (руб)'] < 1_000) | (df_clean['ЗП (руб)'] > 1_000_000)).sum()

In [ ]:
# удаляем выбросы
df_clean.drop(df_clean[(df_clean['ЗП (руб)'] < 1_000) | (df_clean['ЗП (руб)'] > 1_000_000)].index, inplace=True)
df_clean.reset_index(drop=True, inplace=True)

print(f"Осталось резюме: {len(df_clean)}")

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# Преобразуем опыт из месяцев в годы
df_clean['Опыт (лет)'] = df_clean['Опыт работы (месяц)'] / 12.0

# Найдём резюме, где опыт (в годах) > возраста
invalid_mask = df_clean['Опыт (лет)'] > df_clean['Возраст']

# Выведем количество таких записей
print(f"Количество резюме с опытом > возраста: {invalid_mask.sum()}")

In [ ]:
# Удалим выбросы
df_clean.drop(df_clean[invalid_mask].index, inplace=True)

# Удалим временный столбец
df_clean.drop(columns=['Опыт (лет)'], inplace=True)
df_clean.reset_index(drop=True, inplace=True)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
log_age = np.log(df_clean['Возраст'])

# Статистики
mean_log = log_age.mean()
std_log = log_age.std()

# Построение гистограммы
fig = px.histogram(
    x=log_age,
    nbins=200,
    title='Логарифмическое распределение возраста<br>Распределение смещено влево (отрицательная асимметрия)',
    labels={'x': 'log(Возраст)', 'count': 'Частота'}
)

# Линии: среднее и границы
fig.add_vline(
    x=mean_log,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Среднее: {mean_log:.2f}"
)

fig.add_vline(
    x=mean_log - 3 * std_log,
    line_dash="dash",
    line_color="orange",
    annotation_text=f"-3: {mean_log - 3 * std_log:.2f}"
)

fig.add_vline(
    x=mean_log + 3 * std_log,
    line_dash="dash",
    line_color="orange",
    annotation_text=f"+3: {mean_log + 3 * std_log:.2f}"
)

fig.add_vline(
    x=mean_log + 4 * std_log,  # послабление вправо
    line_dash="dash",
    line_color="orange",
    annotation_text=f"+4 (послабление): {mean_log + 4 * std_log:.2f}"
)

fig.show()

In [ ]:
z_scores = (log_age - mean_log) / std_log

# Выбросы: z < -3 или z > 4
valid_mask = (z_scores >= -3) & (z_scores <= 4)
print(f"Количество выбросов (по z-отклонению): {(~valid_mask).sum()}")

outliers_df = df_clean[~valid_mask][['Возраст']].copy()
print(f"Аномальные возрасты:\n {outliers_df}")

In [ ]:
# удаление выбросов из данных
df_clean = df_clean[valid_mask].reset_index(drop=True)

Вывод:\
в выбросы попали соискатели с возрастом 15 и 100 лет.